In [ ]:
# Parameters: 
politician_id <- "240"
aggregation <- "day" # aggregation can be by day ('day'), week ('week') or month('month')
from_date <-'2017-09-01'
to_date <-'2018-12-31'

In [ ]:
library("httr")
library("jsonlite")
library("ggplot2")
library('RJSplot')
library('dplyr')

## Using API

In [ ]:
base <- "193.175.238.88:8080/api/politicians/"
call1 <- paste(base,"all/", sep="")
get_politicians_df <- as.data.frame(fromJSON(content(GET(call1), "text", encoding="UTF-8"), flatten = TRUE))
call_chobs <- paste(base,"wikipedia/chobs/politicians/", politician_id, "/?from_date=",from_date,"&to_date=",to_date,"&aggregate_by=", aggregation, sep="")
chobs = as.data.frame(fromJSON(content(GET(call_chobs), "text", encoding="UTF-8"), flatten = TRUE))

#extracting revisions, conflicts and undos via wikiwho API
base_ww <- "https://www.wikiwho.net/de/edit_persistence/v1.0.0-beta/page/"
wiki_id <- get_politicians_df$wp_ids[get_politicians_df$politician_id == politician_id]
wiki = as.data.frame(fromJSON(content(GET(paste(base_ww, wiki_id, "/?start=",from_date,"&end=",to_date , sep="")), "text", encoding="UTF-8"), flatten = TRUE))
wiki$undos <- wiki$editions.dels + wiki$editions.reins
wiki <- summarise(group_by(wiki, editions.year_month), undos = sum(undos),revisions = sum(editions.revisions), conflicts = sum(editions.conflicts))



In [ ]:
page_id<-get_politicians_df[politician_id,c('wp_ids')]
call_page<-paste("https://de.wikipedia.org/w/api.php?action=query&prop=info&pageids=",page_id,"&inprop=url&format=json", sep="")
page = as.data.frame(fromJSON(content(GET(call_page), "text", encoding="UTF-8"), flatten = TRUE))
wiki$editions.year_month<-as.POSIXct(strptime(wiki[["editions.year_month"]], format='%Y-%m-%d'),tz = 'GMT')
wiki_title=page[, 4]
call_views <- paste("https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/", wiki_title,"/", "daily/",gsub("-", "", from_date),"/",gsub("-", "", to_date), sep="")
views = as.data.frame(fromJSON(content(GET(call_views), "text", encoding="UTF-8"), flatten = TRUE))
views<-views[c("items.timestamp", "items.views")]
views$items.timestamp<-as.POSIXct(strptime(views[["items.timestamp"]], format='%Y%m%d%H'),tz = 'GMT')

if (aggregation  == 'day'){
    views<-summarise(group_by(views, time = cut(items.timestamp, "day")), items.views = sum(items.views))
    }
if (aggregation  == 'week'){
    views<-summarise(group_by(views, time = cut(items.timestamp, "week")), items.views = sum(items.views))
    }
if (aggregation  == 'month'){  
    views<-summarise(group_by(views, time = cut(items.timestamp, "month")), items.views = sum(items.views))
    }

wiki_data = merge( views, chobs, by.x='time', by.y='labels', all.x = TRUE)

In [ ]:
#plotting
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = wiki_data, mapping = aes(as.Date(time))) +
geom_line(aes(y = values, color="Change objects")) +
  geom_line(aes(y = items.views/100, color="Views")) +    
   labs(title = "Wikipedia Activity", x = "", y = "Change Objects") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank()) +
  scale_x_date(date_breaks = "1 month") + scale_y_continuous(sec.axis = sec_axis(~ . *100, name = "Views"))
#plotting
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = wiki, mapping = aes(as.Date(editions.year_month))) +
  geom_line(aes(y = revisions, color="Revisions")) +  
   labs(title = "Wikipedia Activity", x = "", y = "Revisions") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank()) +
  scale_x_date(date_breaks = "1 month")

In [ ]:
#plotting
options(repr.plot.width=8, repr.plot.height=4)
ggplot(data = wiki, mapping = aes(as.Date(editions.year_month))) +
  geom_line(aes(y = undos, color="Undos")) +
  geom_line(aes(y = conflicts, color="Conflicts")) +
  labs(title = "Wikipedia conflicts", x = "", y = "Undos and Conflicts") +
  theme(axis.text.x = element_text( size = 7, angle = 60, hjust = 0.5, vjust = 0.5),
        axis.text.y = element_text( size = 7), legend.title = element_blank()) +
  scale_x_date(date_breaks = "1 month")
